<a href="https://colab.research.google.com/github/Utkarshp1/Bayesian_Optimisation/blob/master/Custom_BO_loop_with_BoTorch_and_Ax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a custom BoTorch model with Ax

In [1]:
!pip install gpytorch

     |████████████████████████████████| 501kB 23.8MB/s 


In [2]:
!pip install botorch

     |████████████████████████████████| 481kB 28.3MB/s 


In [3]:
!pip3 install ax-platform

     |████████████████████████████████| 819kB 28.5MB/s 


## Set up the optimization problem in Ax

We optimize the Branin function, a simple benchmark function in two dimensions. This function is defined as follows:
$$ f(x) = \biggl(x_{2} - \frac{5.1}{4\pi^{2}}x_{1}^{2} + \frac{5}{\pi}x_{1}-6\biggr)^2 + 10\biggl(1-\frac{1}{8\pi}\biggr)cos(x_{1}) + 10$$

Search Space: $x_{1} \in [-5, 10]$ and $x_{2} \in [0, 15]$

In [26]:
import random
import numpy as np

def branin(parameterization, *args):
    x1, x2 = parameterization["x1"], parameterization["x2"]
    y = (x2 - 5.1 / (4*np.pi**2) * x1**2 + 5*x1/np.pi - 6)**2
    y += 10*(1- 1/(8*np.pi))*np.cos(x1) + 10
    # let's add some synthetic observation noise
    # y += random.normalvariate(0, 0.1)
    return {'branin': (y, 0.0)}

We need to define a search space for our experiment that defines the parameters and the set of feasible values.

In [27]:
from ax import ParameterType, RangeParameter, SearchSpace

search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name="x1", parameter_type=ParameterType.FLOAT, lower=-5, upper=10
        ),
        RangeParameter(
            name="x2", parameter_type=ParameterType.FLOAT, lower=0, upper=15
        ),
    ]
)

Third, we make a `SimpleExperiment`-- note that the `objective_name` needs to be one of the metric names returned by the evaluation function.

In [28]:
from ax import SimpleExperiment

exp = SimpleExperiment(
    name='test_branin',
    search_space=search_space,
    evaluation_function=branin,
    objective_name="branin",
    minimize=True,
)

## Implementing the custom model (Gaussian Process Model)

We implement a very simple gpytorch Exact GP Model that uses an RBF kernel (with ARD) and infers a (homoskedastic) noise level.

ARD means Each input dimension gets its own separate lengthscale (i.e. $\Theta$ in the RBF kernel is a non-constant diagonal matrix).

In [29]:
from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import GammaPrior

In [30]:
class SimpleCustomGP(ExactGP, GPyTorchModel):
    _num_outputs = 1    # to inform GPyTorchModel API

    def __init__(self, X_train,  y_train):
        # squeeze output dim before passing y_train to ExactGP
        super().__init__(X_train, y_train.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=RBFKernel(ard_num_dims=X_train.shape[-1])
        )
        self.to(X_train)    # make sure we're on the right device/dtype

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

## Define a factory function to be used with Ax's BotorchModel

In [31]:
from botorch.fit import fit_gpytorch_model

def _get_and_fit_simple_custom_gp(Xs, Ys, **kwargs):
    model = SimpleCustomGP(Xs[0], Ys[0])
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    return model

We use the Sobol generator to create 5 (quasi-) random initial point in the search space. Calling `batch_trial` will cause Ax to evaluate the underlying `branin` function at the generated points, and automatically keep track of the results.

In [32]:
from ax.modelbridge import get_sobol

sobol = get_sobol(exp.search_space)
exp.new_batch_trial(generator_run=sobol.gen(5))

BatchTrial(experiment_name='test_branin', index=0, status=TrialStatus.CANDIDATE)

## Run the optimization loop

In [37]:
from ax.modelbridge.factory import get_botorch

for i in range(10):
    print(f"Running optimization batch {i+1}/10...")
    model = get_botorch(
        experiment=exp,
        data=exp.eval(),
        search_space=exp.search_space,
        model_constructor=_get_and_fit_simple_custom_gp,
    )
    batch = exp.new_trial(generator_run=model.gen(1))

Running optimization batch 1/10...
Running optimization batch 2/10...
Running optimization batch 3/10...
Running optimization batch 4/10...
Running optimization batch 5/10...
Running optimization batch 6/10...
Running optimization batch 7/10...
Running optimization batch 8/10...
Running optimization batch 9/10...
Running optimization batch 10/10...


In [34]:
data = exp.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,branin,1.282987,0.0,0
1,0_1,branin,46.457799,0.0,0
2,0_2,branin,26.361503,0.0,0
3,0_3,branin,150.215032,0.0,0
4,0_4,branin,13.832162,0.0,0
5,1_0,branin,250.447089,0.0,1
6,2_0,branin,7.718688,0.0,2
7,3_0,branin,2.416050,0.0,3
8,4_0,branin,61.640575,0.0,4
9,5_0,branin,206.794755,0.0,5


In [36]:
data = exp.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,branin,1.282987,0.0,0
1,0_1,branin,46.457799,0.0,0
2,0_2,branin,26.361503,0.0,0
3,0_3,branin,150.215032,0.0,0
4,0_4,branin,13.832162,0.0,0
5,1_0,branin,250.447089,0.0,1
6,2_0,branin,7.718688,0.0,2
7,3_0,branin,2.416050,0.0,3
8,4_0,branin,61.640575,0.0,4
9,5_0,branin,206.794755,0.0,5


In [38]:
data = exp.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,branin,1.282987,0.0,0
1,0_1,branin,46.457799,0.0,0
2,0_2,branin,26.361503,0.0,0
3,0_3,branin,150.215032,0.0,0
4,0_4,branin,13.832162,0.0,0
5,1_0,branin,250.447089,0.0,1
6,2_0,branin,7.718688,0.0,2
7,3_0,branin,2.416050,0.0,3
8,4_0,branin,61.640575,0.0,4
9,5_0,branin,206.794755,0.0,5
